## **Modelisation**

### **Decision Tree**

**Decision Tree avec class_weight= balanced sans réglage d'hyperparamètres0.605**

In [ ]:

Y = train["isFraud"]
X = train.drop(columns='isFraud')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45)

In [ ]:
base_tree = DecisionTreeClassifier(class_weight='balanced', random_state = 42)
base_tree.fit(X_train, Y_train)


base_tree_train_predict = base_tree.predict_proba(X_train)
base_tree_test_predict = base_tree.predict_proba(X_test)

print('base random forest train auc:', roc_auc_score(Y_train, base_tree_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_tree_test_predict[:,1]))

base random forest train auc: 1.0
base random forest test auc: 0.7148519107400491


In [ ]:
Y_final = base_tree.predict_proba(test)
submission_Tree = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud': Y_final[:,1]})


In [ ]:
filename = 'final8.csv'
submission_Tree.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: final8.csv


**Decision Tree avec class_weight= balanced avec réglage d'hyperparamètres 0.831675**

In [ ]:

Y = train["isFraud"]
X = train.drop(columns='isFraud')
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45)

In [ ]:
# Function to measure the quality of a split
criterion = ['gini', 'entropy']
# Strategy used to choose the split at each node, random selects a set of features randomly and splits,
splitter = ['best', 'random']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 210, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 30, 50, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10, 20]
# The number of features to consider when looking for the best split
max_features = ['auto', 'log2', None]
# Maximum number of leaf nodes
max_leaf_nodes = [int(x) for x in np.linspace(10, 210, num = 11)]
[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000 , 12000, 15000, None]
# Split threshold of a decrease of the impurity
min_impurity_decrease = [0, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.0015, 0.002, 0.005, 0.01, 0.02]
# Complexity parameter used for Minimal Cost-Complexity Pruning
ccp_alpha = [x for x in np.linspace(0, 0.05, num = 11)]
# Create the random grid
random_grid = {'criterion': criterion,
               'splitter': splitter,
               'max_depth': max_depth,               
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'max_leaf_nodes': max_leaf_nodes,
               'min_impurity_decrease': min_impurity_decrease,
               'ccp_alpha': ccp_alpha}

# pprint(random_grid)

In [ ]:
base_tree = DecisionTreeClassifier(class_weight='balanced', random_state = 42)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune: base_tree
# Random search of parameters with auc_roc scoring, using 3 fold stratified cross validation,
skf = StratifiedKFold(n_splits=3, random_state=42)
# search across 60 different combinations, and use all available cores
tree_random = RandomizedSearchCV(estimator = base_tree, param_distributions = random_grid, n_iter = 60, cv = skf, scoring = 'roc_auc', verbose=0, random_state=42, n_jobs = -1)
# Fit the random search model
tree_random.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
                   error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight='balanced',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf...
                                        'max_leaf_nodes': [10, 30, 50, 70,

In [ ]:
best_random_tree_train_predict = tree_random.predict_proba(X_train)
best_random_tree_test_predict = tree_random.predict_proba(X_test)
print('best random tree train auc:', roc_auc_score(Y_train, best_random_tree_train_predict[:,1]))
print('best random tree test auc:', roc_auc_score(Y_test, best_random_tree_test_predict[:,1]))

best random tree train auc: 0.8224246259854169
best random tree test auc: 0.8277030415985582


In [ ]:
Y_final = tree_random.predict_proba(test)

In [ ]:
submission_tree = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud': Y_final[:,1]})

In [ ]:
filename = 'final10.csv'
submission_tree.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: final10.csv


**Decision Tree avec Resample sans réglage d'hyperparamètres 0,693844**

In [ ]:

# concatenate training data back together
# separate minority and majority class
not_fraud = train[train.isFraud==0]
fraud = train[train.isFraud==1]
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27)
# combine minority and downsampled majority
train = pd.concat([not_fraud_downsampled, fraud])    # Concatenation
# let's check the classes counts
train.isFraud.value_counts()

1    20663
0    20663
Name: isFraud, dtype: int64

In [ ]:

Y = train["isFraud"]
X = train.drop(columns='isFraud')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45)

In [ ]:
base_tree = DecisionTreeClassifier(class_weight='balanced', random_state = 42)
base_tree.fit(X_train, Y_train)


base_tree_train_predict = base_tree.predict_proba(X_train)
base_tree_test_predict = base_tree.predict_proba(X_test)

print('base random forest train auc:', roc_auc_score(Y_train, base_tree_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_tree_test_predict[:,1]))

base random forest train auc: 1.0
base random forest test auc: 0.7449588643329569


In [ ]:
Y_final = base_tree.predict_proba(test)
submission_Tree2 = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud': Y_final[:,1]})


In [ ]:
filename = 'final9.csv'
submission_Tree2.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: final9.csv


**Decision Tree avec Resample avec réglage d'hyperparamètres 0,839628**

In [ ]:

# concatenate training data back together
# separate minority and majority class
not_fraud = train[train.isFraud==0]
fraud = train[train.isFraud==1]
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27)
# combine minority and downsampled majority
train = pd.concat([not_fraud_downsampled, fraud])    # Concatenation
# let's check the classes counts
train.isFraud.value_counts()

1    20663
0    20663
Name: isFraud, dtype: int64

In [ ]:

Y = train["isFraud"]
X = train.drop(columns='isFraud')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45)

In [ ]:
# Function to measure the quality of a split
criterion = ['gini', 'entropy']
# Strategy used to choose the split at each node, random selects a set of features randomly and splits,
splitter = ['best', 'random']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 210, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 30, 50, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 10, 20]
# The number of features to consider when looking for the best split
max_features = ['auto', 'log2', None]
# Maximum number of leaf nodes
max_leaf_nodes = [int(x) for x in np.linspace(10, 210, num = 11)]
[1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000 , 12000, 15000, None]
# Split threshold of a decrease of the impurity
min_impurity_decrease = [0, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.0015, 0.002, 0.005, 0.01, 0.02]
# Complexity parameter used for Minimal Cost-Complexity Pruning
ccp_alpha = [x for x in np.linspace(0, 0.05, num = 11)]
# Create the random grid
random_grid = {'criterion': criterion,
               'splitter': splitter,
               'max_depth': max_depth,               
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features,
               'max_leaf_nodes': max_leaf_nodes,
               'min_impurity_decrease': min_impurity_decrease,
               'ccp_alpha': ccp_alpha}

# pprint(random_grid)

In [ ]:
base_tree = DecisionTreeClassifier(class_weight='balanced', random_state = 42)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune: base_tree
# Random search of parameters with auc_roc scoring, using 3 fold stratified cross validation,
skf = StratifiedKFold(n_splits=3, random_state=42)
# search across 60 different combinations, and use all available cores
tree_random = RandomizedSearchCV(estimator = base_tree, param_distributions = random_grid, n_iter = 60, cv = skf, scoring = 'roc_auc', verbose=0, random_state=42, n_jobs = -1)
# Fit the random search model
tree_random.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
                   error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight='balanced',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf...
                                        'max_leaf_nodes': [10, 30, 50, 70,

In [ ]:
best_random_tree_train_predict = tree_random.predict_proba(X_train)
best_random_tree_test_predict = tree_random.predict_proba(X_test)
print('best random tree train auc:', roc_auc_score(Y_train, best_random_tree_train_predict[:,1]))
print('best random tree test auc:', roc_auc_score(Y_test, best_random_tree_test_predict[:,1]))

best random tree train auc: 0.8211494760679858
best random tree test auc: 0.8183711754475644


In [ ]:
Y_final = tree_random.predict_proba(test)

In [ ]:
submission_tree4 = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud': Y_final[:,1]})

In [ ]:
filename = 'final12.csv'
submission_tree4.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: final12.csv


### **Random Forest**

**RandomForest avec class_weight="balanced" et sans réglage d'hyperparamètres 0.854356**

In [ ]:
Y=train["isFraud"]
X=train.drop(columns=["isFraud"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45,stratify=Y)

In [ ]:
base_rf = RandomForestClassifier(class_weight='balanced', random_state=42)
base_rf.fit(X_train, Y_train)
base_rf_train_predict = base_rf.predict_proba(X_train)
base_rf_test_predict = base_rf.predict_proba(X_test)

print('base random forest train auc:', roc_auc_score(Y_train, base_rf_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_rf_test_predict[:,1]))

base random forest train auc: 1.0
base random forest test auc: 0.9025413904520481


In [ ]:
RanForestBD=base_rf.predict_proba(test)[:,1] 

In [ ]:
FinalRBD = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud':RanForestBD})

In [ ]:
filename = 'FinalRBD.csv'

FinalRBD.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: FinalRBD.csv


**RandomForest avec class_weight="balanced" et réglage des hyperparamétres 0.887373**

In [ ]:
Y=train["isFraud"]
X=train.drop(columns=["isFraud"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=45,stratify=Y)

In [ ]:
base_rf = RandomForestClassifier(class_weight='balanced', random_state=42)


In [ ]:
# Number of trees in random forest
n_estimators = [10,20,30]
# Function to measure the quality of a split
criterion = ['gini', 'entropy']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 30, 50, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [20,30,40]
# The number of features to consider when looking for the best split
max_features = [0.7,'auto', 'log2', None]
random_grid = {'n_estimators': n_estimators,
               'criterion': criterion,               
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features
               
              }

# pprint(random_grid)

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
# Use the random grid to search for best hyperparameters
# First create the base model to tune: base_rf
# Random search of parameters with auc_roc scoring, using 3 fold stratified cross validation,
skf = StratifiedKFold(n_splits=3, random_state=42)
# search across 60 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = base_rf, param_distributions = random_grid, n_iter = 60, 
                               cv = skf, scoring = 'roc_auc', verbose=0, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight='balanced',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_sample...
                   ii

In [ ]:
print(rf_random.best_estimator_.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 80, 'max_features': 0.7, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 20, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 30, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
rf_random = RandomForestClassifier(bootstrap= True, ccp_alpha= 0.0, class_weight='balanced', criterion= 'gini', max_depth= 80, max_features= 0.7, max_leaf_nodes= None, max_samples= None, min_impurity_decrease= 0.0, min_impurity_split= None, min_samples_leaf= 20, min_samples_split= 5, min_weight_fraction_leaf= 0.0, n_estimators= 30, n_jobs= None, oob_score= False, random_state= 42, verbose= 0, warm_start= False)
rf_random.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=80, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
base_rf_train_predict = rf_random.predict_proba(X_train)
base_rf_test_predict = rf_random.predict_proba(X_test)

print('base random forest train auc:', roc_auc_score(Y_train, base_rf_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_rf_test_predict[:,1]))

base random forest train auc: 0.9957077369610208
base random forest test auc: 0.9114323529288872


In [ ]:
RanForestBR=rf_random.predict_proba(test)[:,1] 

In [ ]:
FinalRBR = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud':RanForestBR})

In [ ]:
filename = 'FinalRBR.csv'

FinalRBR.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: FinalRBR.csv


**RandomForest avec Resample et sans réglage des hyperparamétres 0.892719**

In [ ]:
from sklearn.utils import resample
# concatenate training data back together
# separate minority and majority class
not_fraud = train[train.isFraud==0]
fraud = train[train.isFraud==1]
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27)
# combine minority and downsampled majority
train = pd.concat([not_fraud_downsampled, fraud])    # Concatenation
# let's check the classes counts
train.isFraud.value_counts()

1    20663
0    20663
Name: isFraud, dtype: int64

In [ ]:
Y=train["isFraud"]
X=train.drop(columns=["isFraud"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [ ]:
base_rf= RandomForestClassifier(random_state=42)

In [ ]:
base_rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:

base_rf_train_predict = base_rf.predict_proba(X_train)
base_rf_test_predict = base_rf.predict_proba(X_test)

print('base random forest train auc:', roc_auc_score(Y_train, base_rf_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_rf_test_predict[:,1]))

base random forest train auc: 1.0
base random forest test auc: 0.8910293157742585


In [ ]:
RanForestRSR=base_rf.predict_proba(test)[:,1] 

In [ ]:
FinalRSR = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud':RanForestRSR})

In [ ]:
filename = 'FinalRSR.csv'

FinalRSR.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: FinalRSR.csv


**RandomForest avec Resample et réglage des hyperparamètres 0.901086**

In [ ]:
# concatenate training data back together
# separate minority and majority class
not_fraud = train[train.isFraud==0]
fraud = train[train.isFraud==1]
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27)
# combine minority and downsampled majority
train = pd.concat([not_fraud_downsampled, fraud])    # Concatenation
# let's check the classes counts
train.isFraud.value_counts()

1    20663
0    20663
Name: isFraud, dtype: int64

In [ ]:
Y=train["isFraud"]
X=train.drop(columns=["isFraud"])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42)

In [ ]:
base_rf= RandomForestClassifier(random_state=42)

In [ ]:
# Number of trees in random forest
n_estimators = [10,20,30]
# Minimum number of samples required at each leaf node
min_samples_leaf = [20,30,40]
# The number of features to consider when looking for the best split
max_features = [0.7,'auto', 'log2', None]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'min_samples_leaf': min_samples_leaf,
               'max_features': max_features

}
# pprint(random_grid)

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
# Use the random grid to search for best hyperparameters
# First create the base model to tune: base_rf
# Random search of parameters with auc_roc scoring, using 3 fold stratified cross validation,
skf = StratifiedKFold(n_splits=3, random_state=42)
# search across 60 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = base_rf, param_distributions = random_grid, n_iter = 60, 
                               cv = skf, scoring = 'roc_auc', verbose=0, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 36 is smaller than n_iter=60. Running 36 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_spli...
                     

In [ ]:
base_rf_train_predict = rf_random.predict_proba(X_train)
base_rf_test_predict = rf_random.predict_proba(X_test)
print('base random forest train auc:', roc_auc_score(Y_train, base_rf_train_predict[:,1]))

print('base random forest test auc:', roc_auc_score(Y_test, base_rf_test_predict[:,1]))

base random forest train auc: 0.9560577953634031
base random forest test auc: 0.8891405530362251


In [ ]:
print(rf_random.best_estimator_.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 0.7, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 20, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 30, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [ ]:
rf_random= RandomForestClassifier(bootstrap= True, ccp_alpha= 0.0, class_weight=None, criterion= 'gini', max_depth= None, max_features= 0.7, max_leaf_nodes= None, max_samples= None, min_impurity_decrease=0.0, min_impurity_split= None, min_samples_leaf= 20,min_samples_split= 2, min_weight_fraction_leaf= 0.0, n_estimators=30, n_jobs= None, oob_score= False, random_state= 42, verbose= 0, warm_start= False)
rf_random.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [ ]:
FinalRAR=rf_random.predict_proba(test)[:,1] 

In [ ]:
RanForestRAR = pd.DataFrame({'TransactionID':test['TransactionID'],'isFraud':FinalRAR})

In [ ]:
filename = 'FinalRAR.csv'

RanForestRAR.to_csv(filename,index=False)

print('Saved file: ' + filename)

Saved file: FinalRAR.csv
